# 10 years of Ericsson Patents Analysed

## Imports and parameter selection
Analysis can be done for any compagny from a start date onward.
First step is to download the data files from the USPTO at: http://www.patentsview.org/download/ .
We will need the assignee, inventor, patent, patent_assignee and patent_inventor.

In [3]:
import pandas as pd
from datetime import datetime
import hashlib

compagny = 'Ericsson'
start_date = '2006-01-01'

## Import assignee data and Filter
Load in assignee and filter to reduce size.
Keep only the ones matching the organization of interest.

In [4]:
assignee = pd.read_csv('assignee.tsv', 
                       dtype={'id': 'str', 
                              'type': 'float', # since some NaN 
                              'name_first': 'str', 
                              'name_last': 'str', 
                              'organization': 'str'}, 
                       sep='\t', 
                       encoding='latin_1')
assignee = assignee[assignee.organization.str.contains(compagny).fillna(value=False)]
print('We now consider {} assignee related to {}.'.format(len(assignee),compagny))
assignee.head(4)

We now consider 61 assignee related to Ericsson.


,id,type,name_first,name_last,organization
7257,04b906651f577930baee601f138176a8,3.0,NaN,NaN,Telefonaktiebolaget LM Ericsson (publ)
15056,09f1e45a46f55d3049ef23fbce9f4cfd,3.0,NaN,NaN,Telafonaktiebolaget LM Ericsson
24153,0ff2dad8213e59c2b302e9d1d719b03a,2.0,NaN,NaN,Ericsson Messaging Systems Inc.
27361,1200a07f7cf63b5f12831db703b03350,2.0,NaN,NaN,"Anaconda-Ericsson, Inc."


In [5]:
# We might have to further filter the organizations if some are deemed not part of the scope.
assignee.organization.unique()

array(['Telefonaktiebolaget LM Ericsson (publ)',
       'Telafonaktiebolaget LM Ericsson',
       'Ericsson Messaging Systems Inc.', 'Anaconda-Ericsson, Inc.',
       'Ericsson Enterprise GmbH', 'ST-Ericsson PVT. Ltd.',
       'Ericsson Austria AG', 'Ericsson Mobile Communications AB',
       'ST-Ericsson (France) SAS',
       'Ericsson GE Mobile Communications Inc.',
       'Telfonaktiebolaget LM Ericsson (publ)',
       'Lefonaktiebolaget LM Ericsson (publ)',
       'Telefonaktiebolaget LN Ericsson',
       'Telefonaktieblaget L M Ericsson',
       'Relefonaktiebolaget L M Ericsson',
       'Telefon Aktie Bolaget LM Ericsson',
       'Ericsson Technology Licensing AB',
       'Ericsson Telecomunicacoes S.A.',
       'Ericsson Invention Ireland Limited',
       'Ericsson Business Networks AB', 'Ericsson-LG Enterprise Co., Ltd.',
       'Ericsson Inc. Research', 'Teefonaktiebolaget L M Ericsson (Publ)',
       'Telefonaktiegolaget IM Ericsson                                            

## Import patent assignee
Load in patent assignee and filter to reduce size.
Choose only patent which have one of the patent assignee for the organization of interest.

In [6]:
patent_assignee = pd.read_csv('patent_assignee.tsv', 
                              dtype={'patent_id' : 'str',
                                     'assignee_id' : 'str'},
                              sep='\t', 
                              encoding='latin_1')
patent_assignee = patent_assignee[patent_assignee.assignee_id.isin(assignee.id.unique())]
print('We now consider {} patents/assignee related to {}.'.format(len(patent_assignee),compagny))
print('{} patents.'.format(len(patent_assignee.patent_id.unique())))
print('{} assignees.'.format(len(patent_assignee.assignee_id.unique())))
patent_assignee.head(4)

We now consider 17261 patents/assignee related to Ericsson.
17244 patents.
61 assignees.


,patent_id,assignee_id
35262,9436453,cab8bea421bb2dac405fd3d14099851a
35263,9497188,cab8bea421bb2dac405fd3d14099851a
121205,9489109,89ab9855fcd847494ed5ee0b97bca89a
124260,9407534,239c99670a20bca25395649d9cfa9f96


## Import patent inventors
Load in patent inventors and filter to reduce size.
Choose only inventors for which the patent is assigned to an assignee for the organization of interest.

In [7]:
patent_inventor = pd.read_csv('patent_inventor.tsv',
                              dtype={'patent_id' : 'str',
                                     'inventor_id' : 'str'},
                              sep='\t', 
                              encoding='latin_1')
patent_inventor = patent_inventor[patent_inventor.patent_id.isin(patent_assignee.patent_id.unique())]
print('We now consider {} patents/inventors related to {}.'.format(len(patent_inventor),compagny))
print('{} patents.'.format(len(patent_inventor.patent_id.unique())))
print('{} inventors.'.format(len(patent_inventor.inventor_id.unique())))
patent_inventor.head(4)

We now consider 38648 patents/inventors related to Ericsson.
17244 patents.
10748 inventors.


,patent_id,inventor_id
112,6897736,5802458-1
286,9300460,8446886-5
295,8626086,8005439-1
642,9520961,7561586-3


## Import inventors details (names)
Load in the inventors details and filter to reduce size.
Keep only the inventors selected above.

In [8]:
inventor = pd.read_csv('inventor.tsv', 
                       dtype={'id' : 'str',
                              'name_first' : 'str',
                              'name_last' : 'str'},
                       sep='\t', 
                       encoding='latin_1')
inventor = inventor[inventor.id.isin(patent_inventor.inventor_id.unique())]
print('We now consider {} inventors related to {}.'.format(len(inventor),compagny))
inventor.head(4)

We now consider 10748 inventors related to Ericsson.


,id,name_first,name_last
2340,3931633-1,John M.,Shannon
3573,3932408-2,Frank A.,Muller
3866,3932550-2,Takahiro,Sato
6329,3934093-1,Per-Olof,Thyselius


## Import the patent details
Load in the patent information and filter to reduce size.
Keep only the patents selected above.

In [9]:
iter_csv = pd.read_csv('patent.tsv',
                       dtype={'id' : 'str',
                              'type' : 'str',
                              'number' : 'str',
                              'country' : 'str',
                              'date' : datetime,
                              'abstract' : 'str',
                              'title' : 'str',
                              'kind' : 'str',
                              'num_claims' : 'int',
                              'filename' : 'str'},
                       iterator=True, 
                       chunksize=100000, # In case of memory exhaustion, chuncksize can be reduced.
                       sep='\t', 
                       encoding='latin_1')
patent = pd.concat([chunk[chunk.id.isin(patent_assignee.patent_id.unique())] for chunk in iter_csv])
print('We now consider {} patents related to {}.'.format(len(patent),compagny))
patent.head(4)

We now consider 17244 patents related to Ericsson.


,id,type,number,country,date,abstract,title,kind,num_claims,filename
3818,3934093,utility,3934093,US,1976-01-20,The present invention relates to a transit ex...,Transit exchange for asynchronous data with un...,A,8,pftaps19760120_wk03.txt
6337,3936612,utility,3936612,US,1976-02-03,In a relay matrix there are provided a plural...,Control and holding circuit for relay matrices,A,11,pftaps19760203_wk05.txt
6506,3936781,utility,3936781,US,1976-02-03,Selecting device for crossbar switches of the...,Selecting device for crossbar switches,A,10,pftaps19760203_wk05.txt
7750,3938025,utility,3938025,US,1976-02-10,A DC to DC converter which gives two individu...,DC to DC converter for producing two individua...,A,5,pftaps19760210_wk06.txt


## Limit the scope of patents after a specified date

In [10]:
patentfromdate = patent[patent.date>=start_date]
patent_inventorfromdate = patent_inventor[patent_inventor.patent_id.isin(patentfromdate.id.unique())]
print('We now consider {} patents/inventors related to {} from {}.'.format(len(patent_inventorfromdate), compagny, start_date))
print('{} patents.'.format(len(patent_inventorfromdate.patent_id.unique())))
print('{} inventors.'.format(len(patent_inventorfromdate.inventor_id.unique())))
patent_inventorfromdate.head(4)

We now consider 26222 patents/inventors related to Ericsson from 2006-01-01.
10856 patents.
7023 inventors.


,patent_id,inventor_id
286,9300460,8446886-5
295,8626086,8005439-1
642,9520961,7561586-3
665,8861623,6452524-2


## Limit the scope of inventors for patents after a specified date

In [48]:
inventorfromdate = inventor[inventor.id.isin(patent_inventorfromdate.inventor_id.unique())]
print('We now consider {} inventors related to {} from {}.'.format(len(inventorfromdate), compagny, start_date))
inventor.head(4)

We now consider 7023 inventors related to Ericsson from 2006-01-01.


,id,name_first,name_last
2340,3931633-1,John M.,Shannon
3573,3932408-2,Frank A.,Muller
3866,3932550-2,Takahiro,Sato
6329,3934093-1,Per-Olof,Thyselius


## Build the edges table for Gephi
Now we need an inventor to inventor table. The edges of the relationship graph.
For performance reasons we keep only one direction of the graph (non directed graph).

In [90]:
edges = pd.DataFrame(columns=('Source', 'Target'))
idx = 0
for p_id in patent_inventorfromdate.patent_id.unique():
    df = patent_inventorfromdate[patent_inventorfromdate.patent_id==p_id]
    for i_id in df.inventor_id.unique():
        for j_id in df.inventor_id.unique():
            if i_id != j_id:
                m = hashlib.md5()
                m.update(i_id.encode('utf-8'))
                i = int(m.hexdigest(),16)
                m = hashlib.md5()
                m.update(j_id.encode('utf-8'))
                j = int(m.hexdigest(),16)
                if i>j:
                    edges.loc[idx] = [i_id, j_id]
                    idx += 1
edges['Type'] = 'Undirected'
edges.head(4)

,Source,Target,Type
0,8446886-5,9300460-5,Undirected
1,8446886-5,8649423-2,Undirected
2,8446886-5,9300460-4,Undirected
3,8446886-5,6347542-1,Undirected


## Build the node table for Gephi

In [93]:
patentcount = patent_inventorfromdate.groupby('inventor_id').patent_id.count()
connectioncount = pd.DataFrame({'id': pd.concat([edges.Source,edges.Target])}).groupby('id').id.count()
nodes = inventorfromdate.copy()
nodes['Label']=inventorfromdate.name_first+' '+inventorfromdate.name_last
nodes['Weight'] = nodes['id'].apply(lambda x: patentcount[x])
nodes['Connections'] = nodes['id'].apply(lambda x: connectioncount[x] if x in connectioncount.index else 0)
del nodes['name_first']
del nodes['name_last']
print('Weight (number of patents) range from {} to {}.'.format(nodes.Weight.min(), nodes.Weight.max()))
print('(Number of) Connections range from {} to {}.'.format(nodes.Connections.min(), nodes.Connections.max()))
nodes.head(4)

Weight (number of patents) range from 1 to 221.
(Number of) Connections range from 0 to 437.


,id,Label,Weight,Connections
2340,3931633-1,John M. Shannon,2,2
3866,3932550-2,Takahiro Sato,1,8
8661,3935643-1,Michael Russell,1,1
12138,3938040-1,Bengt G. Lofmark,1,0


## Save the results

In [92]:
with open('edges_{}.csv'.format(compagny), 'w') as f:
    edges.to_csv(f)
with open('nodes_{}.csv'.format(compagny), 'w') as f:
    nodes.to_csv(f)